In [0]:
print("Meghana Sharma")

Meghana Sharma


In [0]:
import requests

url = "https://google-map-places2.p.rapidapi.com/textsearch"

querystring = {"query":"restaurants in Sydney"}

headers = {
    "x-rapidapi-key": "519ceeac66msh6d200392f76ed01p1c235djsn3e4f3a996932",
    "x-rapidapi-host": "google-map-places2.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())


{'html_attributions': [], 'next_page_token': 'AeeoHcJUTaqqfbM7nDgw8saiJA5qvAl0ef4-Y4irImAJvYICx4t2Sg6wkcKkJo_yoQK3QRekAGaAPjJeo2dlPr_TgBG5YT7CECvYMSpCD54Vt4k0pXqij1NQOxagQixVQhbvUxKkmUwG5wTyctvgLTg7W7jdodTAib4bpLf5DfYjQpFdlkwCent_z6LwHiD1yEDP14CYhiET5IimWel9kPP5XIqwqAoxUvUF6p6oZjP3uO2RNtGAm5JkSnApTE34FcLwYykMD2g6jZiOvwlCnCKQTyDD7Ww9giyz86Xb1NjId8PPnQMxgogW-3Pee_pGftqgn304jCVXI2P7Cauwbj6llFHBTWmBwi72rPvZm2S_J5ZWr0D0wxuHItw0j123TkU8DOe5S5eWCSXTsRnJDGntqwyIzA', 'results': [{'business_status': 'OPERATIONAL', 'formatted_address': 'Shop 7.07-7.08/25 Martin Pl, Sydney NSW 2000, Australia', 'geometry': {'location': {'lat': -33.8680725, 'lng': 151.2093505}, 'viewport': {'northeast': {'lat': -33.86693242010728, 'lng': 151.2108943298927}, 'southwest': {'lat': -33.86963207989272, 'lng': 151.2081946701073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'icon_background_color': '#FF9E67', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/

In [0]:
#Define all lib at once
import requests
import json
import time 
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# API configuration 
url = "https://google-map-places2.p.rapidapi.com/textsearch"
headers = {
    "x-rapidapi-key": "519ceeac66msh6d200392f76ed01p1c235djsn3e4f3a996932",
    "x-rapidapi-host": "google-map-places2.p.rapidapi.com"
}

# Coordinates for Seneca Global(Search in Google maps->right click on Red button->What's here)
latitude = 17.434848
longitude = 78.376755

# EXTRACT: Fetch data from API
querystring = {
    "query": "cafe OR coffee", #Searching for cafe 
    "location": f"{latitude},{longitude}",
    "radius": "200"  # in meters-Give minimum radius as our cafe is in the same building
}

response = requests.get(url, headers=headers, params=querystring)
data = response.json()

# TRANSFORM: Process and Structure Data
cafes = []
for place in data.get("results", []):  # Keeping this loop, since it processes individual cafes
    cafe_data = {
        "name": place.get("name"),
        "address": place.get("formatted_address", place.get("vicinity", "N/A")),
        "rating": str(place.get("rating", "N/A")),
        "latitude": place["geometry"]["location"]["lat"],
        "longitude": place["geometry"]["location"]["lng"]
    }
    cafes.append(cafe_data)
# Convert Processed Data into Pandas DataFrame
df = pd.DataFrame(cafes)


# LOAD : Save data to Spark Table

schema = StructType([
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("rating", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
])

df_spark = spark.createDataFrame(df, schema=schema)
df_spark.write.mode("append").saveAsTable("Nearby_SenecaGlobalCafes")
display(df_spark)


name,address,rating,latitude,longitude
Qaffeine,"Unnamed Road, Silpa Gram Craft Village, HITEC City, Rai Durg, Hyderabad, Telangana 500081, India",4.4,17.4335028,78.3771782
BREWS & BLENDS - T-Hub,"T-Hub 2-0 | Technology Hub, 2-0 20, Inorbit Mall Rd, Vittal Rao Nagar, Madhapur, Hyderabad, Telangana 500081, India",4.6,17.4338791,78.3791092
Dreams Cafe,"ground floor, The pinnacle, Lumbini Avenue, Gachibowli, Hyderabad, Telangana 500032, India",4.8,17.4328347,78.3739506
House of Truly (Gachibowli),"Ground Floor, Plot No.44, Jayabheri Pine Valley, Gachibowli, Hyderabad, Telangana 500032, India",4.7,17.4304425,78.3716366
Third Wave Coffee,"Gate Number 6, Retail Ground Floor, Magma Block Circle 20, Knowledge City, Salarpuria Sattva, Raidurg Village, Serilingampalle (M), Hyderabad, Telangana 500032, India",4.6,17.4330773,78.3823398
TRUE BLACK speciality coffee | The Loft - Hitech City,"THE LOFT, Silpa Gram Craft Village, HITEC City, Hyderabad, Telangana 500081, India",4.5,17.4322944,78.3825981
Brew La La,"1, Hitech City Main Rd, near Galaxy by Auro Realty, Lumbini Avenue, Gachibowli, Hyderabad, Telangana 500081, India",4.6,17.4368021,78.37544489999999
Lubb dubb cafe,"Care Hospitals, Jayabheri Pine Valley, Gachibowli, Hyderabad, Telangana 500032, India",4.3,17.4306196,78.3720754
Roast - The Caffeine Capital,"82&84, Hitech City Main Rd, near Deccan serai, HUDA Techno Enclave, HITEC City, Hyderabad, Telangana 500081, India",4.4,17.4427945,78.3824664
abCoffee The Skyview RMZ,"RMZ The Skyview, Food Court, 1st Floor, The Skyview, 20, Hitech City Main Rd, Diamond Hills, Lumbini Avenue, Gachibowli, Rai Durg, Hyderabad, Telangana 500032, India",5,17.430678,78.3756326


In [0]:
%sql
select * from Nearby_SenecaGlobalCafes



name,address,rating,latitude,longitude
Qaffeine,"Unnamed Road, Silpa Gram Craft Village, HITEC City, Rai Durg, Hyderabad, Telangana 500081, India",4.4,17.4335028,78.3771782
BREWS & BLENDS - T-Hub,"T-Hub 2-0 | Technology Hub, 2-0 20, Inorbit Mall Rd, Vittal Rao Nagar, Madhapur, Hyderabad, Telangana 500081, India",4.6,17.4338791,78.3791092
Dreams Cafe,"ground floor, The pinnacle, Lumbini Avenue, Gachibowli, Hyderabad, Telangana 500032, India",4.8,17.4328347,78.3739506
House of Truly (Gachibowli),"Ground Floor, Plot No.44, Jayabheri Pine Valley, Gachibowli, Hyderabad, Telangana 500032, India",4.7,17.4304425,78.3716366
Third Wave Coffee,"Gate Number 6, Retail Ground Floor, Magma Block Circle 20, Knowledge City, Salarpuria Sattva, Raidurg Village, Serilingampalle (M), Hyderabad, Telangana 500032, India",4.6,17.4330773,78.3823398
TRUE BLACK speciality coffee | The Loft - Hitech City,"THE LOFT, Silpa Gram Craft Village, HITEC City, Hyderabad, Telangana 500081, India",4.5,17.4322944,78.3825981
Brew La La,"1, Hitech City Main Rd, near Galaxy by Auro Realty, Lumbini Avenue, Gachibowli, Hyderabad, Telangana 500081, India",4.6,17.4368021,78.37544489999999
Lubb dubb cafe,"Care Hospitals, Jayabheri Pine Valley, Gachibowli, Hyderabad, Telangana 500032, India",4.3,17.4306196,78.3720754
Roast - The Caffeine Capital,"82&84, Hitech City Main Rd, near Deccan serai, HUDA Techno Enclave, HITEC City, Hyderabad, Telangana 500081, India",4.4,17.4427945,78.3824664
abCoffee The Skyview RMZ,"RMZ The Skyview, Food Court, 1st Floor, The Skyview, 20, Hitech City Main Rd, Diamond Hills, Lumbini Avenue, Gachibowli, Rai Durg, Hyderabad, Telangana 500032, India",5,17.430678,78.3756326


In [0]:
%sql
---select * from Nearby_SenecaGlobalCafes
----where name like '%Nilo%'
select * from Nearby_SenecaGlobalCafes
where LOWER(name) LIKE '%nilo%'

name,address,rating,latitude,longitude
Cafe Niloufer Hitech City,"Plot no.30/C, Sy No.83/1, Tower - 2, Orbit Building, TSIIC, Raidurgam, Penmaktha, Serilingampalle (M), Hyderabad, Telangana 500081, India",4,17.4341926,78.37713099999999
